# Scrap repo stats using PyGitHub

### Author: Crystal Zang

This notebook utilized GitHub access tokens (PAT) to scrape GitHub repository statistics such as stargazers, watchers, forks, and topics. One PAT would scrape at a rate of 5000 repositories per hour. Utilizing 36 PATs we would scrape 10,288,063 repositories in about XXX hours at a rate of 15,514 repositories per hour.

#### Warnings
You should not commit any access topen to GitHub, which would result in access token being revoked.


In [128]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [129]:
# load packages 
import os
import psycopg2 as pg
from sqlalchemy import create_engine
import pandas as pd
import requests as r
import string 
import json
import base64
import urllib.request
import itertools 
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from github import Github, RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException
import warnings
import datetime

import multiprocessing
#from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import Pool, freeze_support

import concurrent.futures

warnings.simplefilter(action='ignore', category=FutureWarning)

In [130]:
#os.environ['db_user'] = ''
#os.environ['db_pwd'] = ''

# connect to the database, download data, limit to repos with at least 20,000 commits?
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

raw_slug_data = '''SELECT * FROM gh_2007_2020.repos_ranked WHERE (commits BETWEEN '990' AND '1000')'''
#raw_slug_data = '''SELECT * FROM gh_2007_2020.repos_ranked WHERE commits < 1000'''

# convert to a dataframe, show how many missing we have (none)
raw_slug_data = pd.read_sql_query(raw_slug_data, con=connection)
print(raw_slug_data.head())
print(raw_slug_data.shape)
print(raw_slug_data.isna().sum())

#get rid of leading and ending space, save slugs to a list
raw_slugs = raw_slug_data["slug"].tolist()
slugs = []
for s in raw_slugs:
    slugs.append(s.strip())  
print(len(slugs))
print(slugs[0], slugs[len(slugs)-1])

#PATs access token, saved as a dataframe
github_pats = '''SELECT * FROM gh_2007_2020.pats_update'''
github_pats = pd.read_sql_query(github_pats, con=connection)

#PATs access token, saved as a list
access_tokens = github_pats["token"]

#number of tokens available for use, a numeric value
num_token = '''SELECT COUNT(*) FROM gh_2007_2020.pats_update'''
num_token = pd.read_sql_query(num_token, con=connection)
num_token=num_token.iloc[0]['count']

connection.close()

                                 id        spdx                   slug  \
0  MDEwOlJlcG9zaXRvcnkzMjQ4MzYxNg==  Apache-2.0          l337ch/kraken   
1  MDEwOlJlcG9zaXRvcnkzMDkzNjE2OTM=    AGPL-3.0       iBringit/algo-01   
2  MDEwOlJlcG9zaXRvcnkxOTY5NzAxNjE=     GPL-2.0      rma945/CryptoRatt   
3  MDEwOlJlcG9zaXRvcnkxMTQ1NTE2NDg=         MIT  mikedennis/FullNodeUI   
4  MDEwOlJlcG9zaXRvcnkxMTExODYwODA=         MIT         ignacio-vc/stu   

            createdat                                        description  \
0 2015-03-18 20:50:03                                               None   
1 2020-11-02 12:22:33                                               None   
2 2019-07-15 09:42:38  Password management solution for a DevOps engi...   
3 2017-12-17 16:31:51                                               None   
4 2017-11-18 07:34:40  Fork of st (simple terminal) by suckless, just...   

  primarylanguage                                        branch  commits  \
0             HCL  MDM

In [103]:
# index ranges from 0 to maximum number of PATs available
def get_access_token(github_pat_index):
    if github_pat_index < num_token:
       # print("Extracting access token #", github_pat_index+1,", total", num_token, "tokens are available.")
        return github_pats.token[github_pat_index]
    else:
        print("token exceed limit")

In [131]:
#practice on 7 slugs, one invalid slug 
slugs_example = ["moderndive/ModernDive_book", "DSPG-Young-Scholars-Program/dspg21oss", 
                 "unknownrepo", "moderndive/ModernDive_book", "lsst/ip_diffim", "esrlabs/chipmunk", "paulmillr/chokidar"]

In [132]:
len(access_tokens)

28

# Not using any multiprocessing

In [133]:
def pull_repo_stats(github_pat_index, slugs):
    df_repo_stats = pd.DataFrame()
    for slug in slugs:
        if github_pat_index >= len(access_tokens):
            github_pat_index -= len(access_tokens)
            print("***Pat access token exceed limit, restart access token loop with #", github_pat_index)
        while github_pat_index < len(access_tokens):
            try:
                access_token = get_access_token(github_pat_index)
                #print("Scrapping --", slug,". Extracting access token #", github_pat_index+1,", total", num_token, "tokens are available.")
                #if false, retry until true, max number of retry is 20 times
                g = Github(access_token, retry = 20, timeout = 15)
                repo = g.get_repo(slug)
                df_repo_stats = df_repo_stats.append({
                    "slug": slug,
                    'stars': repo.stargazers_count,
                    'watchers': repo.subscribers_count,
                    'forks': repo.forks_count,
                    'topics': repo.get_topics()
                }, ignore_index = True)
            except RateLimitExceededException as e:
                print(e.status)
                print('Rate limit exceeded --', slug, ", using access token #", github_pat_index)
                print("Current time:", datetime.datetime.now())
                #time.sleep(300)
                github_pat_index+=1
                print("***Exit current access token, proceed with next aceess token #", github_pat_index, "rescrape --",slug)
                break
            except BadCredentialsException as e:
                print(e.status)
                print('Bad credentials exception --', slug, ", using access token #", github_pat_index)
                print("Current time:", datetime.datetime.now())
                github_pat_index+=1
                print("***Exit current access token, proceed with next aceess token #", github_pat_index, "rescrape --",slug)
                break
            except UnknownObjectException as e:
                print(e.status)
                print('Unknown object exception --', slug)
                break
            except GithubException as e:
                print(e.status)
                print('General exception --', slug)
                break
            except r.exceptions.ConnectionError as e:
                print('Retries limit exceeded --', slug)
                print(str(e))
                time.sleep(10)
                continue
            except r.exceptions.Timeout as e:
                print('Time out exception --', slug)
                print(str(e))
                time.sleep(10)
                continue
            break
    return df_repo_stats



In [11]:
#df_repo_stats.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_stats_bt_1000_1500.csv', index = False)   


# Function to scrape one slug using specified pat index

In [142]:
#global github_pat_index
def pull_repo_stats(slug, github_pat_index):
    #github_pat_index = 1
   # if github_pat_index >= len(access_tokens):
   #     github_pat_index -= len(access_tokens)
   #     print("***Pat access token exceed limit, restart access token loop with #", github_pat_index)
    while True:
        try:
            if github_pat_index >= len(access_tokens):
                github_pat_index -= len(access_tokens)
                print(f"***Pat access token exceed limit, restart access token loop with #", github_pat_index)  
                
            access_token = get_access_token(github_pat_index)
            #print("Scrapping --", slug,". Extracting access token #", github_pat_index+1,", total", num_token, "tokens are available.")
            #if false, retry until true, max number of retry is set to 3 times
            g = Github(access_token, retry = 3, timeout = 15)
            repo = g.get_repo(slug)
            stars = repo.stargazers_count
            watchers = repo.subscribers_count
            forks = repo.forks_count
            topics = repo.get_topics()
        except RateLimitExceededException as e:
            print(e.status)
            print('WARNING: Rate limit exceeded --', slug, ", using access token #", github_pat_index)
            #time.sleep(300)
            github_pat_index+=1
            print("***Exit current access token, proceed with next aceess token #", github_pat_index, "rescrape --",slug)
            break
        except BadCredentialsException as e:
            print(e.status)
            print('WARNING: Bad credentials exception --', slug, ", using access token #", github_pat_index)
            github_pat_index+=1
            print("***Exit current access token, proceed with next aceess token #", github_pat_index, "rescrape --",slug)
            break
        except UnknownObjectException as e:
            print(e.status)
            print('WARNING: Unknown object exception --', slug)
            stars = None
            watchers = None
            forks =  None
            topics = None
            return slug, stars, watchers, forks, topics
            break
        except GithubException as e:
            print(e.status)
            print('General exception --', slug)
            break
        except r.exceptions.ConnectionError as e:
            print('Retries limit exceeded --', slug)
            print(str(e))
            time.sleep(10)
            continue
        except r.exceptions.Timeout as e:
            print('Time out exception --', slug)
            print(str(e))
            time.sleep(10)
            continue
            
        results = (slug, stars, watchers, forks, topics)
        #return slug, stars, watchers, forks, topics
        return results

In [143]:
# Example
i = 0
print(slugs_example[i])
result = pull_repo_stats(slugs_example[i], 0)
print(result[3])

#print(result[0])
#print(result[1] ==None)

moderndive/ModernDive_book


NameError: name 'get_access_token' is not defined

## Multiprocessing approach: pool.apply_async for multiple parameters

In [136]:
def get_result(results):
    global slug_log
    global stars_log
    global watchers_log
    global forks_log
    global topics_log
    if result is None:
        slug_log.append(None)
        stars_log.append(None)
        watchers_log.append(None)
        forks_log.append(None)
        topics_log.append(None)  
    else:
        slug_log.append(results[0])
        stars_log.append(results[1])
        watchers_log.append(results[2])
        forks_log.append(results[3])
        topics_log.append(results[4])
        #print("appending results")

In [137]:
def get_stars(stars):
    global stars_log
    stars_log.append(stars)
    print("appending results")

In [140]:
stars_log = []
#practice on 2 slugs, one invalid slug 
#slugs_example = ["moderndive/ModernDive_book", "unknownrepo"]

#practice on 7 slugs, one invalid slug 
slugs_example = ["moderndive/ModernDive_book", "DSPG-Young-Scholars-Program/dspg21oss", 
                 "unknownrepo", "moderndive/ModernDive_book", "lsst/ip_diffim", "esrlabs/chipmunk", "paulmillr/chokidar"]


slugs_100 = slugs[0:100]

In [141]:
# Approach 2. pool.apply_async for multiple parameters
if __name__ == '__main__':
    cores_available = multiprocessing.cpu_count() - 1
    print(f'There are {cores_available} CPUs available.')
    #pool = multiprocessing.Pool(cores_available)
    pool = multiprocessing.Pool(4)
    
    # now we will feed in all of the remaining slugs 
    slug_log = []
    stars_log = []
    watchers_log = []
    forks_log = []
    topics_log = []

    start_time = datetime.datetime.now()
    print("Start scraping. Start time:", start_time)
    
    for slug in slugs:
        #get_result(pull_repo_stats(slug, 0))
        
        pool.apply_async(pull_repo_stats, args=(slug,  3),callback = get_result)
        #result = pool.apply_async(pull_repo_stats, args=(slugs_example, 3)).get()
        
        print("~~~~~~~~~~~~~~~slug: ",slug, "using access token #", github_pat_index, "~~~~~~~~~~~~~~~~")
        #if result is None:
         #   slug_log.append(None)
         #   stars_log.append(None)
         #   watchers_log.append(None)
         #   forks_log.append(None)
         #   topics_log.append(None)  
        #else:
         #   slug_log.append(result[0])
         #   stars_log.append(result[1])
         #   watchers_log.append(result[2])
         #   forks_log.append(result[3])
         #   topics_log.append(result[4])
    pool.close()
    pool.join()
    
    final_log = pd.DataFrame({'slug': slug_log, "stars": stars_log, 'watchers': watchers_log, 'forks': forks_log, 'topics': topics_log}, columns=["slug", "stars", "watchers", "forks", "topics"])

    end_time = datetime.datetime.now()
    print("Finished scraping", len(final_log), "of", len(slugs_example), "records at", end_time)
    print("It took", end_time-start_time, "to run.")

There are 39 CPUs available.
Start scraping. Start time: 2021-06-24 16:53:54.715233


NameError: name 'github_pat_index' is not defined

In [115]:
#print(final_log)
final_log

,slug,stars,watchers,forks,topics
0,unknownrepo,NaN,NaN,NaN,None
1,moderndive/ModernDive_book,548.0,28.0,295.0,"[moderndive, data-science, tidyverse, statisti..."
2,moderndive/ModernDive_book,548.0,28.0,295.0,"[moderndive, data-science, tidyverse, statisti..."
3,DSPG-Young-Scholars-Program/dspg21oss,0.0,3.0,0.0,[]
4,lsst/ip_diffim,5.0,45.0,7.0,[]
5,esrlabs/chipmunk,285.0,11.0,23.0,"[logs-analysis, logviewer, search, logstash, l..."
6,paulmillr/chokidar,7990.0,85.0,511.0,"[watch-files, fsevents, watcher, filesystem]"


## Multiprocessing approach: pool.imap_unordered

In [37]:
cores_available = multiprocessing.cpu_count() - 1
print(f'There are {cores_available} cores available.')
pool = Pool(cores_available)

# now we will feed in all of the remaining slugs 
slug_log = []
stars_log = []
watchers_log = []
forks_log = []
topics_log = []

start_time = datetime.datetime.now()
print("Start scraping. Start time:", start_time)
for result in pool.imap_unordered(pull_repo_stats, slugs_example):
    print("~~~~~~~~~~~~~~~result: ",result, "~~~~~~~~~~~~~~~~")
    if result is None:
        slug_log.append(None)
        stars_log.append(None)
        watchers_log.append(None)
        forks_log.append(None)
        topics_log.append(None)  
    else:
        slug_log.append(result[0])
        stars_log.append(result[1])
        watchers_log.append(result[2])
        forks_log.append(result[3])
        topics_log.append(result[4])


final_log = pd.DataFrame({'slug': slug_log, "stars": stars_log, 'watchers': watchers_log, 'forks': forks_log, 'topics': topics_log}, columns=["slug", "stars", "watchers", "forks", "topics"])

end_time = datetime.datetime.now()
print("Finished scraping", len(final_log), "of", len(slugs), "records at", end_time)
print("It took", end_time-start_time, "to run.")


There are 39 cores available.
404
Start scraping. Start time: 2021-06-23 11:22:11.104201
~~~~~~~~~~~~~~~result:  ('rrrrrrrrr', None, None, None, None) ~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~result:  ('DSPG-Young-Scholars-Program/dspg21oss', 0, 3, 0, []) ~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~result:  ('moderndive/ModernDive_book', 548, 28, 295, ['moderndive', 'data-science', 'tidyverse', 'statistical-inference', 'r', 'ggplot2', 'infer', 'hypothesis-testing', 'confidence-intervals', 'regression', 'regression-models', 'data-visualization', 'data-wrangling', 'tidy', 'rstudio', 'rstats', 'dplyr', 'bootstrap-method', 'permutation-test']) ~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~result:  ('moderndive/ModernDive_book', 548, 28, 295, ['moderndive', 'data-science', 'tidyverse', 'statistical-inference', 'r', 'ggplot2', 'infer', 'hypothesis-testing', 'confidence-intervals', 'regression', 'regression-models', 'data-visualization', 'data-wrangling', 'tidy', 'rstudio', 'rstats', 'dplyr', 'bootstrap-method', 'permutation-t

In [38]:
final_log

,slug,stars,watchers,forks,topics
0,rrrrrrrrr,NaN,NaN,NaN,None
1,DSPG-Young-Scholars-Program/dspg21oss,0.0,3.0,0.0,[]
2,moderndive/ModernDive_book,548.0,28.0,295.0,"[moderndive, data-science, tidyverse, statisti..."
3,moderndive/ModernDive_book,548.0,28.0,295.0,"[moderndive, data-science, tidyverse, statisti..."


In [309]:
#final_log.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/final_log_try.csv', index = False)   

In [317]:
# read in the file and check
print(final_log.head())
print(final_log)
print(final_log.isna().sum())
print(final_log.shape)

                                    slug  stars  watchers  forks  \
0                              rrrrrrrrr    NaN       NaN    NaN   
1  DSPG-Young-Scholars-Program/dspg21oss    0.0       3.0    0.0   
2             moderndive/ModernDive_book  548.0      28.0  295.0   
3             moderndive/ModernDive_book  548.0      28.0  295.0   

                                              topics  
0                                               None  
1                                                 []  
2  [moderndive, data-science, tidyverse, statisti...  
3  [moderndive, data-science, tidyverse, statisti...  
                                    slug  stars  watchers  forks  \
0                              rrrrrrrrr    NaN       NaN    NaN   
1  DSPG-Young-Scholars-Program/dspg21oss    0.0       3.0    0.0   
2             moderndive/ModernDive_book  548.0      28.0  295.0   
3             moderndive/ModernDive_book  548.0      28.0  295.0   

                                           